# Exercise 2: RAG

Goal: Implement a simple Retrieval Augmented Generation (RAG) system.


In [ ]:
%pip install databricks-langchain langchain-community pymupdf
%restart_python

In [ ]:
import glob
import os

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import InMemoryVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from llm import embeddings as embed
from llm import model

In [ ]:
def create_vectorstore():
    pdf_folder = "data/knowledge_base"
    documents = []

    if not os.path.exists(pdf_folder):
        print(f"Warning: {pdf_folder} not found.")
        return None

    # Exercise 2.1: Load all PDFs from the folder
    # Hint: Use `PyMuPDFLoader(file).load()`
    for file in glob.glob(f"{pdf_folder}/*.pdf"):
        # <solution>
        # TODO: Implement this
        pass
        # </solution>

    if not documents:
        print("No documents found.")
        return None

    # Exercise 2.2: Split documents
    # Hint: Use `RecursiveCharacterTextSplitter`
    # <solution>
    # TODO: Implement this
    pass
    # </solution>

    # Exercise 2.3: Create vectorstore.
    # Hint: Use the `InMemoryVectorStore` constructor.
    # <solution>
    # TODO: Implement this
    pass
    # </solution>
    return vectorstore

In [ ]:
def retrieve(vectorstore, query: str):
    # Exercise 2.4: Implement a retriever using the vectorstore.
    # <solution>
    # TODO: Implement this
    pass
    # </solution>
    return [doc.page_content for doc in matching_docs]

In [ ]:
def main():
    # Initialize
    vectorstore = create_vectorstore()
    if not vectorstore:
        return

    query = "What surface additives have excellent leveling properties?"
    context = retrieve(vectorstore, query)

    # Exercise 2.5: Generate Answer
    prompt = ChatPromptTemplate.from_template(
        "Answer the question based on the context:\nContext: {context}\nQuestion: {question}"
    )
    # <solution>
    # TODO: Implement this
    pass
    # </solution>
    print(f"Question: {query}")
    print(f"Answer: {response.content}")


if __name__ == "__main__":
    main()
    # Response should include: BYK-388, BYK-3440, and BYK-3441